<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-15 18:22:36--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-15 18:22:36 (79.9 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [7]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
hub_module = hub.KerasLayer(HUB_URL)

rgb_features = hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [8]:
model._name = 'combined-model'

In [9]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 533, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='auc', 
                                              verbose=1,
                                              patience=3,
                                              restore_best_weights=True,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+"-{epoch:02d}.h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [ ]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    class_weight=class_weight)

Epoch 1/20
533/533 [==============================] - 2064s 4s/step - loss: 361.4439 - auc: 0.4187 - accuracy: 0.5714
Epoch 2/20
533/533 [==============================] - 430s 805ms/step - loss: 187.8491 - auc: 0.6002 - accuracy: 0.7286
Epoch 3/20
533/533 [==============================] - 442s 828ms/step - loss: 161.5165 - auc: 0.6642 - accuracy: 0.7735
Epoch 4/20
533/533 [==============================] - 444s 832ms/step - loss: 123.0131 - auc: 0.7067 - accuracy: 0.8033
Epoch 5/20
533/533 [==============================] - 444s 832ms/step - loss: 110.2435 - auc: 0.7285 - accuracy: 0.8197
Epoch 6/20
533/533 [==============================] - 444s 832ms/step - loss: 94.2289 - auc: 0.7614 - accuracy: 0.8418
Epoch 7/20
533/533 [==============================] - 444s 831ms/step - loss: 81.6719 - auc: 0.7742 - accuracy: 0.8499
Epoch 8/20
533/533 [==============================] - 444s 831ms/step - loss: 57.8375 - auc: 0.8236 - accuracy: 0.8831
Epoch 9/20
533/533 [=========================

Training was stopped here so I had to re-train from epoch 20

In [13]:
model = keras.models.load_model('./logs/checkpoints/combined-model-19.h5',
                                custom_objects={'KerasLayer': hub.KerasLayer})

In [14]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    initial_epoch=19,
                    class_weight=class_weight)

Epoch 20/20
533/533 [==============================] - 472s 830ms/step - loss: 5.7539 - auc: 0.9591 - accuracy: 0.9673


In [15]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.2.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.2.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.2.h5

In [17]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=30,
                    initial_epoch=20,
                    class_weight=class_weight)

Epoch 21/30
 61/533 [==>...........................] - ETA: 6:08 - loss: 3.4607 - auc: 0.9695 - accuracy: 0.9749

KeyboardInterrupt: ignored

In [ ]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.2.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.2.h5